## Welcome to my quick web scrape

Here is a small script made to scrape Hockey Refernce's 2018-19 Regular Sesaon scores page. 

- Utilized Beautiful Soup Library
- Filtered down html file to isloate the rows and columns of the table on Hockey Refernce
- Quick Data cleaning and feature engineering to add Home Game column
- Export as .cvs file type

Source: https://www.hockey-reference.com/leagues/NHL_2019_games.html

In [ ]:
from bs4 import BeautifulSoup, Comment
import requests
import pandas as pd 
from datetime import datetime

### Beatiful Soup commands

In [4]:
url = "https://www.hockey-reference.com/leagues/NHL_2019_games.html"
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"
}
pageTree = requests.get(url, headers=headers)
soup = BeautifulSoup(pageTree.text, "html.parser")
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

### Filter down html file

In [6]:
col_headers = [elem.text for elem in soup.findAll('thead')[0].findAll('th')][1:]
data = [[elem.text for elem in row.findAll('td')] for row in soup.findAll('tbody')[0].findAll('tr')]
index = [elem.text for elem in soup.findAll('tbody')[0].findAll('th')]


### Data cleaning and feature engineering

In [12]:
df = pd.DataFrame(data, columns=col_headers, index=index)
df.reset_index(inplace = True)
df.rename(columns = {'index':'Date'}, inplace = True)

In [13]:
df['Home Game'] = [0]*df.index
for i in df.index:
    
    if df.loc[i,'Home'] not in list(df.loc[0:i-1,'Home']):
        df.loc[i,'Home Game'] = 1
    else:
        df.loc[i,'Home Game'] = list(df.loc[0:i,'Home']).count(df.loc[i,'Home'])+1

In [18]:
df

,Date,Visitor,G,Home,G,,Att.,LOG,Notes,Home Game
0,2018-10-03,Anaheim Ducks,5,San Jose Sharks,2,,"17,562",2:25,,1
1,2018-10-03,Montreal Canadiens,2,Toronto Maple Leafs,3,OT,"19,589",2:37,,1
2,2018-10-03,Calgary Flames,2,Vancouver Canucks,5,,"18,870",2:31,,1
3,2018-10-03,Boston Bruins,0,Washington Capitals,7,,"18,506",2:30,,1
4,2018-10-04,Boston Bruins,4,Buffalo Sabres,0,,"19,070",2:35,,1
...,...,...,...,...,...,...,...,...,...,...
1266,2019-04-06,Carolina Hurricanes,4,Philadelphia Flyers,3,,"19,433",2:25,,42
1267,2019-04-06,New York Rangers,4,Pittsburgh Penguins,3,OT,"18,660",2:34,,42
1268,2019-04-06,Colorado Avalanche,2,San Jose Sharks,5,,"17,562",2:21,,42
1269,2019-04-06,Vancouver Canucks,2,St. Louis Blues,3,SO,"17,970",2:35,,42


### Export as .cvs file type

In [23]:
df.to_csv('../NHL attendance/NHL_attendance_df.csv', index = False)